In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df= pd.read_csv('../data/raw/output.csv')

### Traitement des variables inutiles

In [3]:
# 📌 Suppression des variables inutiles
df_clean = df.drop(columns=["date", "street", "statezip", "country"])

# 📌 Encodage de la variable 'city' par le prix moyen
city_mean_price = df_clean.groupby("city")["price"].mean()

# Création d'une nouvelle variable 'city_mean_price'
df_clean["city_mean_price"] = df_clean["city"].map(city_mean_price)

# Suppression de la colonne 'city' (car remplacée)
df_clean = df_clean.drop(columns=["city"])

df_clean.head()


,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,city_mean_price
0,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,420392.364047
1,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,579837.467282
2,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,439492.444648
3,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,847180.662995
4,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,667649.534781


In [4]:

# On verifie s'il y a bien 44 valeurs uniques pour 'city_mean_price'
# et on affiche leur distribution

variable = 'city_mean_price' 
unique_values = df_clean[variable].unique()

print(len(unique_values))
print(df_clean[variable].value_counts().sort_values())


44
city_mean_price
7.450000e+05       1
5.250000e+05       1
4.250000e+05       1
2.850000e+05       2
5.624500e+05       2
2.330000e+05       3
1.194838e+06       4
2.072880e+05       5
2.252333e+05       6
5.140714e+05       7
3.396056e+05       9
1.321945e+06      11
2.046559e+06      11
6.926818e+05      11
5.067931e+05      18
5.087520e+05      22
3.076146e+05      28
4.725569e+05      29
3.082901e+05      29
2.452906e+05      29
6.660467e+05      33
4.814419e+05      33
4.484750e+05      36
4.039941e+05      42
2.962304e+05      43
3.995657e+05      50
3.049925e+05      58
4.474940e+05      66
5.363053e+05      71
3.489472e+05      74
1.123818e+06      86
3.364749e+05      96
6.095650e+05     115
4.203924e+05     123
2.898877e+05     148
6.869176e+05     175
2.993404e+05     176
4.394924e+05     185
6.515836e+05     187
5.961637e+05     187
6.676495e+05     235
8.471807e+05     286
3.770410e+05     293
5.798375e+05    1573
Name: count, dtype: int64


### Verifications de la plage de la variable binaire waterfront

In [5]:
# Afficher les valeurs et leurs fréquences
print("Distribution de 'waterfront':")
print(df_clean['waterfront'].value_counts().sort_index())

Distribution de 'waterfront':
waterfront
0    4567
1      33
Name: count, dtype: int64


### Les variables conditions et view sont des notes, on verifie leurs distributions pour s'assurer qu'il n y apas de valeurs aberrantes

In [6]:
# Afficher les valeurs et leurs fréquences
print("Distribution de 'condition':")
print(df_clean['condition'].value_counts().sort_index())

# Afficher les valeurs et leurs fréquences
print("Distribution de 'view':")
print(df_clean['view'].value_counts().sort_index())

Distribution de 'condition':
condition
1       6
2      32
3    2875
4    1252
5     435
Name: count, dtype: int64
Distribution de 'view':
view
0    4140
1      69
2     205
3     116
4      70
Name: count, dtype: int64


### Verifions aussi avec les variables bedrooms, bathrooms, et floors

In [7]:
print("Distribution de 'bedrooms':")
print(df_clean['bedrooms'].value_counts().sort_index())

Distribution de 'bedrooms':
bedrooms
0.0       2
1.0      38
2.0     566
3.0    2032
4.0    1531
5.0     353
6.0      61
7.0      14
8.0       2
9.0       1
Name: count, dtype: int64


In [8]:
print("Distribution de 'bathrooms':")
print(df_clean['bathrooms'].value_counts().sort_index())


Distribution de 'bathrooms':
bathrooms
0.00       2
0.75      17
1.00     743
1.25       3
1.50     291
1.75     629
2.00     427
2.25     419
2.50    1189
2.75     276
3.00     167
3.25     136
3.50     162
3.75      37
4.00      23
4.25      23
4.50      29
4.75       7
5.00       6
5.25       4
5.50       4
5.75       1
6.25       2
6.50       1
6.75       1
8.00       1
Name: count, dtype: int64


In [9]:
print("Distribution de 'floors':")
print(df_clean['floors'].value_counts().sort_index())

Distribution de 'floors':
floors
1.0    2174
1.5     444
2.0    1811
2.5      41
3.0     128
3.5       2
Name: count, dtype: int64


### Il y a des maisons à 0 et plus de 5 etages, on verifie si le prix et la surface habitable de ces maisons sont coherants

In [10]:
# Filtrer les maisons avec 0 ou 8 salles de bain
extreme_bath = df_clean[df_clean['bathrooms'].isin([0.00, 8.00])]
print(extreme_bath[['bathrooms','price', 'sqft_living', 'bedrooms']])

      bathrooms      price  sqft_living  bedrooms
122         8.0  2280000.0        13540       7.0
2365        0.0  1095000.0         3064       0.0
3209        0.0  1295648.0         4810       0.0


### Verifions les variables des maisons sans salles de bain

In [11]:
no_bath = df_clean[df_clean['bathrooms'] == 0.00]
no_bath.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,city_mean_price
2365,1095000.0,0.0,0.0,3064,4764,3.5,0,2,3,3064,0,1990,2009,579837.467282
3209,1295648.0,0.0,0.0,4810,28008,2.0,0,0,3,4810,0,1990,2009,667649.534781


### Beaucoup d'incoherances pour ces 2 maisons donc, on fait le choix de les supprimer.

In [12]:
## Suppression des maisons avec 0 salles de bain et reoncstruction du dataframe

df = df[df['bathrooms'] > 0]

df_clean = df.drop(columns=["date", "street", "statezip", "country"])
city_mean_price = df_clean.groupby("city")["price"].mean()
df_clean["city_mean_price"] = df_clean["city"].map(city_mean_price)
df_clean = df_clean.drop(columns=["city"])

df_clean.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,city_mean_price
0,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,420392.364047
1,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,579509.755748
2,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,439492.444648
3,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,847180.662995
4,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,664965.780656


In [13]:
no_bath = df_clean[df_clean['bathrooms'] == 0.00]
no_bath.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,city_mean_price


### Traitement des variables yr_built et yr_renovated

In [14]:
print("Distribution de 'yr_built':")
print(df_clean['yr_built'].value_counts().sort_index())

Distribution de 'yr_built':
yr_built
1900    22
1901     9
1902    10
1903    10
1904     9
        ..
2010    28
2011    24
2012    33
2013    57
2014    78
Name: count, Length: 115, dtype: int64


In [15]:
print("Distribution de 'yr_renovated':")
print(df_clean['yr_renovated'].value_counts().sort_index())

Distribution de 'yr_renovated':
yr_renovated
0       2735
1912      33
1913       1
1923      57
1934       6
1945       7
1948       1
1953       1
1954       8
1955       2
1956      15
1958       5
1960       1
1963      12
1966       1
1968       9
1969      11
1970       9
1971       5
1972      17
1974       6
1975       1
1977       1
1978       3
1979      32
1980       2
1981       1
1982      22
1983      41
1984      10
1985      29
1986       5
1987       1
1988      43
1989      55
1990      16
1991       1
1992      32
1993      39
1994      57
1995       2
1996      22
1997      28
1998      40
1999      41
2000     170
2001     109
2002      41
2003     151
2004      77
2005      95
2006      68
2007       7
2008      45
2009     107
2010      30
2011      54
2012      45
2013      61
2014      72
Name: count, dtype: int64


In [16]:
#Calculer l'âge de la maison par rapport à 2018
df_clean["house_age"] = 2018 - df["yr_built"]

#Variable binaire : rénovée ou non
df_clean["renovated"] = np.where(df["yr_renovated"] > 0, 1, 0)

#Âge depuis la dernière rénovation (si pas rénovée → âge total)
df_clean["age_since_renov"] = np.where(
    df["yr_renovated"] > 0,
    2018 - df["yr_renovated"],
    df_clean["house_age"]
)

#Supprimer les colonnes d'origine
df_clean = df_clean.drop(columns=["yr_built", "yr_renovated"])

df_clean.head()


,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,city_mean_price,house_age,renovated,age_since_renov
0,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,420392.364047,63,1,13
1,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,579509.755748,97,0,97
2,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,439492.444648,52,0,52
3,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,847180.662995,55,0,55
4,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,664965.780656,42,1,26


### Identification des variables ayant une correlation superieure à 0.8

In [17]:

#Calcul de la matrice de corrélation
corr_matrix = df_clean.corr(numeric_only=True)

#On récupère uniquement la partie supérieure (pour éviter les doublons)
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

#Sélection des paires de variables corrélées ≥ 0.8
high_corr_pairs = [
    (col, row, upper.loc[row, col])
    for col in upper.columns
    for row in upper.index
    if abs(upper.loc[row, col]) >= 0.8
]

print("Variables très corrélées entre elles (|corr| ≥ 0.8) :\n")
for var1, var2, corr_val in high_corr_pairs:
    print(f"{var1}  <-->  {var2}   | corr = {corr_val:.2f}")


Variables très corrélées entre elles (|corr| ≥ 0.8) :

sqft_above  <-->  sqft_living   | corr = 0.88


In [18]:
df_clean = df_clean.drop(columns=["sqft_above"])

In [19]:
df_clean.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_basement,city_mean_price,house_age,renovated,age_since_renov
0,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,0,420392.364047,63,1,13
1,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,280,579509.755748,97,0,97
2,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,0,439492.444648,52,0,52
3,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,847180.662995,55,0,55
4,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,800,664965.780656,42,1,26


### Sauvegarde du dataset pour les prochaines etapes

In [20]:
import os
from pathlib import Path


# Chemin relatif pour la sauvegarde
output_path = Path("../data/processed/df_clean.pkl")  # Je recommande 'processed' plutôt qu'external

# Sauvegarde en pickle (conservation des types)
df_clean.to_pickle(output_path)